In [1]:
import sys
from Bio import SeqIO
import getopt
import os
from BCBio import GFF
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
import random
import pandas as pd
import numpy as np

In [ ]:
help = '''
    {script_name} -c com_port [-o output_file] [--loglevel level]

    Reads the temperature data from a radio.  The temperature data is output in csv form.

    examples:
        Read table from radio attached to com4 and write the table to the file
        output.csv.

            {script_name} -c com4 -o output.csv

        Read table from radio attached to com3 and write the table to stdout. 
        You can use IO redirection to send the contents where every you want.

            # just print to the terminal 
            {script_name} -c com3

            # redirect to another file
            {script_name} -c com3 > somefile.csv

            # filter out temperatures that are -100
            {script_name} -c com3 | grep -v '^-100' 


    -c com_port
    --com_port comport
        Name of the COM port attached to the radio

    -o output_file
    --output output_file
        If specified write the table data to the given file.  If not specified
        the data will be written to stdout.

    --loglevel critical | error | warning | info | debug | notset
        Control the verbosity of the script by setting the log level.  Critical
        is the least verbose and notset is the most verbose.

        The default loglevel is {default_loglevel}.

        These values correspond directly to the python logging module levels. 
        (i.e. https://docs.python.org/3/howto/logging.html#logging-levels)


    -h 
    --help 
        print this message

'''

def usage():
    print(help)
    
def rungetopts():
    try:
        opts, args = getopt.getopt(sys.argv[1:], "a:qh", ["accession", "quiet", "help"])
    except getopt.GetoptError as err:
        # print help information and exit:
        print(err) # will print something like "option -a not recognized"
        usage()
        sys.exit(2)
    accession = ""
    for o, a in opts:
            if o in ("-h", "--help"):
                usage()
                sys.exit()
            elif o in ("-a", "--accession"):
                accession = a
            else:
                assert False, "unhandled option"
    if accession == "":
        print("-a <accession> missing. For more help use -h")
        sys.exit(2)
    return(accession)

In [ ]:
def getreaddepths(accession):
    try:
        read_depths_list = {}
        for filename in os.listdir("/Users/thomasnicholson/phd/RNASeq/plot_files/cds_included/%s/" % accession):
            filesize = os.path.getsize("/Users/thomasnicholson/phd/RNASeq/plot_files/cds_included/%s/%s" % (accession, filename))
            if filesize == 0:
                print("No data in %s" % filename)
                continue
            plotFile  = open(os.path.join("/Users/thomasnicholson/phd/RNASeq/plot_files/cds_included/%s/" % accession, filename), 'r')
            print(filename)

            read_depths = []
            for line in plotFile:
                words = line.rstrip()
                words = words.split()
                pos = words[0]
                neg = words[1]
                if pos > neg:
                    read_depths.append(int(pos))
                else:
                    read_depths.append(int(neg))
            read_depths_list[filename] = read_depths
        print("making dataframe")
        df = pd.DataFrame(data = read_depths_list)


        df['mean'] = df.iloc[:].mean(axis=1)
        df['sum'] = df.sum(axis=1)
        df['median'] = df.median(axis=1)
        df['max'] = df.max(axis=1)
        return df
    except IOError:
        print("Cannot open a file in /Users/thomasnicholson/phd/RNASeq/plot_files/cds_included/%s/" % accession)


In [ ]:
def sRNA_read_depths(inFile, read_depths_df,accession):
    outFile  = open("/Users/thomasnicholson/phd/RNASeq/srna_seqs/version_1/read_depths/%s_read_depths.txt" % accession, 'w')
    outFile.write("ID\tstart\tend\tgroup\tfeature\tmean_mean\tmean_median\tmean_max\tmedian_mean\tmedian_median\tmedian_max\tmax_mean\tmax_median\tmax_max\n")
    outFile.close()
    outFile  = open("/Users/thomasnicholson/phd/RNASeq/srna_seqs/version_1/read_depths/%s_read_depths.txt" % accession, 'a')

    for line in inFile:
        words = line.rstrip()
        words = words.split("\t")
        srna = words[-1]
        start = words[2]
        try:
            start = int(start)
        except ValueError:
            continue
        end = words[3]
        end = int(end)
        new_feature = words[8]
        feature = words[1]
        if new_feature == "FALSE":
            srna_type = "known"
        else:
            srna_type = "novel"

        subsetDF = read_depths_df[start:end]

        mean_mean = subsetDF['mean'].mean()
        mean_median = subsetDF['mean'].median()
        mean_max = subsetDF['mean'].max()
        median_mean = subsetDF['median'].mean()
        median_median = subsetDF['median'].median()
        median_max = subsetDF['median'].mean()
        max_mean = subsetDF['max'].mean()
        max_median = subsetDF['max'].median()
        max_max = subsetDF['max'].max()

        outFile.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n" % (srna, start, end, srna_type, feature, mean_mean, mean_median, mean_max, median_mean, median_median, median_max, max_mean, max_median, max_max))

In [ ]:
# accession = rungetopts()
accession = "GCA_000006765.1"
random = True
print(random)
print("Reading files")
try:
    if random == False:
        inFile = open("/Users/thomasnicholson/phd/RNASeq/new_calls/%s_new_calls.txt" % accession, 'r')
    else:
        #inFile = open("/Users/thomasnicholson/phd/RNASeq/new_calls/random/python_version_1/%s_random_new_calls.txt" % accession, 'r')
        print("Running read_csv")
        inDF = pd.read_csv("/Users/thomasnicholson/phd/RNASeq/new_calls/random/python_version_1/%s_random_new_calls.txt" % accession, sep="\t")
except IOError:
    print("/Users/thomasnicholson/phd/RNASeq/new_calls/%s_new_calls.txt not found" % accession)
    sys.exit(2)




In [ ]:
display(inDF)

In [ ]:
read_depths_list = {}
df = None
for filename in os.listdir("/Users/thomasnicholson/phd/RNASeq/plot_files/cds_included/%s/" % accession):
        filesize = os.path.getsize("/Users/thomasnicholson/phd/RNASeq/plot_files/cds_included/%s/%s" % (accession, filename))
        if filesize == 0:
            print("No data in %s" % filename)
            continue
        plotFile  = pd.read_csv(os.path.join("/Users/thomasnicholson/phd/RNASeq/plot_files/cds_included/%s/" % accession, filename), sep = '\t', header=None)
        print(filename)
        plotFile['selected'] = plotFile.iloc[:].max(axis=1)
        tmpDf = plotFile.iloc[:,2]
        if df is not None:
            df = pd.concat([df.reset_index(drop=True), tmpDf], axis=1)
        else:
            df = tmpDf
#         read_depths = []
#         for line in plotFile:
#             words = line.rstrip()
#             words = words.split()
#             pos = words[0]
#             neg = words[1]
#             if pos > neg:
#                 read_depths.append(int(pos))
#             else:
#                 read_depths.append(int(neg))
#         read_depths_list[filename] = read_depths
# print("making dataframe")
# df = pd.DataFrame(data = read_depths_list)

dfOut = df
dfOut['mean'] = df.iloc[:].mean(axis=1)
dfOut['median'] = df.median(axis=1)
dfOut['max'] = df.max(axis=1)





In [ ]:
# display(read_depths_list)
display(dfOut)

In [ ]:
print("getting read depths")
read_depths_df = getreaddepths(accession)


In [ ]:
display(read_depths_df)

In [ ]:
print("writing file")

sRNA_read_depths(inFile, dfOut, accession)

In [68]:
def openNHMMER(nhmmername):
    nhmmerDF = pd.read_csv(nhmmername, delim_whitespace=True, header=None, comment='#')
    nhmmerDF.columns = ["target_name", "accession", "query_name", "accession_2", "hmmfrom", "hmmto", "alifrom", "alito", "envfrom", "envto", "sq_len", "strand", "E_value", "score", "bias", "description_of_target"]
    nhmmerDF[["ID", "descriptors"]] = nhmmerDF.target_name.str.split("[", expand = True)
    nhmmerDF[["ID_2", "descriptors_2"]] = nhmmerDF.query_name.str.split("[", expand = True)
    d = nhmmerDF.groupby('ID')['ID_2'].apply(list).to_dict()
    return(d)

def openReadDepths(readdepthsname, d):
    readdepthsDF = pd.read_csv(readdepthsname, sep = "\t", comment='#')
    readdepthsDF = readdepthsDF[readdepthsDF['ID'] != "ID"]
    readdepthsDF[["mean_value", "mean_decimal"]] = readdepthsDF.max_mean.str.split(".", expand = True)
    readdepthsDF[["median_value", "median_decimal"]] = readdepthsDF.max_median.str.split(".", expand = True)
    readdepthsDF[["max_value", "max_decimal"]] = readdepthsDF.max_max.str.split(".", expand = True)
    readdepthsDF[['mean_value', 'median_value', 'max_value']] = readdepthsDF.loc[:,['mean_value', 'median_value', 'max_value']].apply(pd.to_numeric)
    idList = list(d.keys())
    readdepthsKept = readdepthsDF[readdepthsDF['ID'].isin(idList)]
    return(readdepthsKept)







In [64]:

def writeReadDepths(outname, readDepths, d):
    seen = []
    d2 = {}
    i = 0

    outFile = open(outname, "w")
    outFile.write("ID\tmean_mean\tmean_median\tmean_max\tmedian_mean\tmedian_median\tmedian_max\tmax_mean\tmax_median\tmax_max\tID_2\n")
    outFile.close()
    outFile = open(outname, "a")
    values = []
    for key in d:
    #     print(i)
    #     i += 1
    #     if i > 100:
    #         break
    #     if key in seen:
    #         continue
    #     print(key)
    #     print(seen)
        values = d[key]
        seen.append(values)
        df = readDepths[readDepths['ID'].isin(values)]
        #     print(df['mean_value'].dtypes)
        
    #     print(df['mean_value'].dtypes)
        mean_mean = df['mean_value'].mean()
        mean_median = df['mean_value'].median()
        mean_max = df['mean_value'].max()
        median_mean = df['median_value'].mean()
        median_median = df['median_value'].median()
        median_max = df['median_value'].max()
        max_mean = df['max_value'].mean()
        max_median = df['max_value'].median()
        max_max = df['max_value'].max()
    #     print(key)
    #     print("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n" % (key,mean_mean,mean_median,mean_max,median_mean,median_median,median_max,max_mean,max_median,max_max,values))
        outFile.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n" % (key,mean_mean,mean_median,mean_max,median_mean,median_median,median_max,max_mean,max_median,max_max,values))
    outFile.close()



In [66]:
d = openNHMMER(nhmmername = "/Users/thomasnicholson/phd/RNASeq/srna_seqs/version_1/positive_control.tbl")

In [69]:
readDepthsKept = openReadDepths(readdepthsname  = "/Users/thomasnicholson/phd/RNASeq/srna_seqs/version_1/read_depths.txt", d = d)

In [70]:
writeReadDepths(outname = "/Users/thomasnicholson/phd/RNASeq/srna_seqs/version_1/positive_control_read_depths.txt", readDepths = readDepthsKept, d = d)

In [5]:
def openNHMMER(nhmmername):
    nhmmerDF = pd.read_csv(nhmmername, delim_whitespace=True, header=None, comment='#')
    nhmmerDF.columns = ["target_name", "accession", "query_name", "accession_2", "hmmfrom", "hmmto", "alifrom", "alito", "envfrom", "envto", "sq_len", "strand", "E_value", "score", "bias", "description_of_target"]
    nhmmerDF[["ID", "descriptors"]] = nhmmerDF.target_name.str.split("[", expand = True)
    nhmmerDF[["ID_2", "descriptors_2"]] = nhmmerDF.query_name.str.split("[", expand = True)
    d = nhmmerDF.groupby('ID')['ID_2'].apply(list).to_dict()
    return(d, nhmmerDF)
def getreaddepths(accession):
    try:
        df = None
        for filename in os.listdir("/Users/thomasnicholson/phd/RNASeq/plot_files/%s/" % accession):
            filesize = os.path.getsize(
                "/Users/thomasnicholson/phd/RNASeq/plot_files/%s/%s" % (accession, filename))
            if filesize == 0:
                print("No data in %s" % filename)
                continue
            plotFile = pd.read_csv(
                os.path.join("/Users/thomasnicholson/phd/RNASeq/plot_files/%s/" % accession, filename),
                sep='\t', header=None)
            print(filename)
            plotFile['selected'] = plotFile.iloc[:].max(axis=1)
            tmpDf = plotFile.iloc[:, 2]
            if df is not None:
                df = pd.concat([df.reset_index(drop=True), tmpDf], axis=1)
            else:
                df = tmpDf
        dfOut = df
        dfOut['mean'] = df.iloc[:].mean(axis=1)
        dfOut['median'] = df.median(axis=1)
        dfOut['max'] = df.max(axis=1)
        return dfOut

    except IOError:
        print("Cannot open a file in /Users/thomasnicholson/phd/RNASeq/plot_files/%s/" % accession)

In [6]:
d, nhmmerDF = openNHMMER(nhmmername = "/Users/thomasnicholson/phd/RNASeq/srna_seqs/version_1/predicted.tbl")

In [12]:
plotFiles = getreaddepths("GCA_002355295.1")

SRR3990750_ncRNA.plot
SRR3990749_ncRNA.plot
SRR3990735_ncRNA.plot
SRR3990733_ncRNA.plot
SRR3990734_ncRNA.plot
SRR3990742_ncRNA.plot


In [16]:
display(nhmmerDF)

,target_name,accession,query_name,accession_2,hmmfrom,hmmto,alifrom,alito,envfrom,envto,sq_len,strand,E_value,score,bias,description_of_target,ID,descriptors,ID_2,descriptors_2
0,"GCA_000006765.1_1[101-227,+,novel,ncRNA,1]",-,"GCA_000006765.1_1[101-227,+,novel,ncRNA,1]",-,1,126,1,126,1,126,126,+,4.400000e-35,120.1,1.5,-,GCA_000006765.1_1,"101-227,+,novel,ncRNA,1]",GCA_000006765.1_1,"101-227,+,novel,ncRNA,1]"
1,"GCA_000006765.1_2[8466-8521,+,novel,ncRNA,1]",-,"GCA_000006765.1_2[8466-8521,+,novel,ncRNA,1]",-,1,54,1,54,1,55,55,+,1.000000e-11,44.9,2.1,-,GCA_000006765.1_2,"8466-8521,+,novel,ncRNA,1]",GCA_000006765.1_2,"8466-8521,+,novel,ncRNA,1]"
2,"GCA_000006765.1_7[53652-53741,+,novel,ncRNA,1]",-,"GCA_000006765.1_7[53652-53741,+,novel,ncRNA,1]",-,1,89,1,89,1,89,89,+,8.400000e-23,80.3,1.8,-,GCA_000006765.1_7,"53652-53741,+,novel,ncRNA,1]",GCA_000006765.1_7,"53652-53741,+,novel,ncRNA,1]"
3,"GCA_000006765.1_8[55057-55480,+,novel,ncRNA,0....",-,"GCA_000006765.1_8[55057-55480,+,novel,ncRNA,0....",-,1,422,1,422,1,423,423,+,4.500000e-130,432.7,10.3,-,GCA_000006765.1_8,"55057-55480,+,novel,ncRNA,0.517730496453901]",GCA_000006765.1_8,"55057-55480,+,novel,ncRNA,0.517730496453901]"
4,"GCA_000006765.1_9[56034-56255,+,novel,ncRNA,1]",-,"GCA_000006765.1_9[56034-56255,+,novel,ncRNA,1]",-,1,221,1,221,1,221,221,+,1.800000e-66,223.1,1.4,-,GCA_000006765.1_9,"56034-56255,+,novel,ncRNA,1]",GCA_000006765.1_9,"56034-56255,+,novel,ncRNA,1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71785,"GCA_900243355.1_879[7753320-7753459,+,novel,nc...",-,"GCA_900243355.1_879[7753320-7753459,+,novel,nc...",-,1,139,1,139,1,139,139,+,1.900000e-37,128.4,8.7,-,GCA_900243355.1_879,"7753320-7753459,+,novel,ncRNA,0.76978417266187]",GCA_900243355.1_879,"7753320-7753459,+,novel,ncRNA,0.76978417266187]"
71786,"GCA_000017745.1_1905[4201112-4203303,+,novel,n...",-,"GCA_900243355.1_879[7753320-7753459,+,novel,nc...",-,13,137,2178,2054,2191,2052,2191,-,1.300000e-11,46.1,2.2,-,GCA_000017745.1_1905,"4201112-4203303,+,novel,ncRNA,1]",GCA_900243355.1_879,"7753320-7753459,+,novel,ncRNA,0.76978417266187]"
71787,"GCA_900243355.1_880[7753767-7753860,+,novel,nc...",-,"GCA_900243355.1_880[7753767-7753860,+,novel,nc...",-,1,92,1,92,1,93,93,+,1.200000e-23,83.3,4.1,-,GCA_900243355.1_880,"7753767-7753860,+,novel,ncRNA,1]",GCA_900243355.1_880,"7753767-7753860,+,novel,ncRNA,1]"
71788,"GCA_000017745.1_1905[4201112-4203303,+,novel,n...",-,"GCA_900243355.1_880[7753767-7753860,+,novel,nc...",-,3,91,1741,1653,1743,1651,2191,-,4.100000e-06,28.6,1.2,-,GCA_000017745.1_1905,"4201112-4203303,+,novel,ncRNA,1]",GCA_900243355.1_880,"7753767-7753860,+,novel,ncRNA,1]"


In [28]:
newCalls = pd.read_csv("/Users/thomasnicholson/phd/RNASeq/new_calls/GCA_002355295.1_new_calls.txt", sep = "\t")

In [29]:
display(newCalls)

,sequence,feature,start,end,strand,file_names,row_numbers,prop_overlap,new_feature,number_of_rnaseq_files,score,id
0,NZ_CP023465.1,ncRNA,1462,2053,+,"SRR3990733_sra_calls.gff,SRR3990734_sra_calls....","1,2,3,4,5,6,7,8,9,10,11,12",1.000000,True,12,7,GCA_002355295.1_1
1,NZ_CP023465.1,ncRNA,5075,5242,-,"SRR3990735_sra_calls.gff,SRR3990750_sra_calls....","13,14,15,16,17",1.000000,True,5,2,GCA_002355295.1_2
2,NZ_CP023465.1,ncRNA,14319,14514,+,"SRR3990742_sra_calls.gff,SRR3990750_sra_calls....","18,19,20,21,22,23,24",0.856410,True,7,3,GCA_002355295.1_3
3,NZ_CP023465.1,ncRNA,19048,19330,+,"SRR3990735_sra_calls.gff,SRR3990749_sra_calls....","25,26,27,28,29,30",1.000000,True,6,2,GCA_002355295.1_4
4,NZ_CP023465.1,ncRNA,22748,22887,-,"SRR3990750_sra_calls.gff,SRR3990735_sra_calls....","31,32,33,34,35,36,37,38",1.000000,True,8,4,GCA_002355295.1_5
...,...,...,...,...,...,...,...,...,...,...,...,...
576,NZ_CP023465.1,ncRNA,6183676,6183910,+,"SRR3990750_sra_calls.gff,SRR3990735_sra_calls....","3321,3322,3323,3324,3325",1.000000,True,5,2,GCA_002355295.1_577
577,NZ_CP023465.1,ncRNA,6188032,6188136,-,"SRR3990735_sra_calls.gff,SRR3990734_sra_calls.gff","3326,3327",0.913462,True,2,3,GCA_002355295.1_578
578,NZ_CP023465.1,ncRNA,6196182,6196368,+,"SRR3990735_sra_calls.gff,SRR3990750_sra_calls....","3328,3329,3330,3331,3332,3333,3334,3335,3336,3337",1.000000,True,10,2,GCA_002355295.1_579
579,NZ_CP023465.1,ncRNA,6244744,6244848,-,"SRR3990734_sra_calls.gff,SRR3990749_sra_calls....","3338,3339,3340,3341,3342,3343",1.000000,True,6,6,GCA_002355295.1_580


In [30]:
idName = "GCA_002355295.1_131"
values = d[idName]
df = newCalls[newCalls['id'].isin(values)]


In [31]:
display(df)

,sequence,feature,start,end,strand,file_names,row_numbers,prop_overlap,new_feature,number_of_rnaseq_files,score,id
2,NZ_CP023465.1,ncRNA,14319,14514,+,"SRR3990742_sra_calls.gff,SRR3990750_sra_calls....","18,19,20,21,22,23,24",0.856410,True,7,3,GCA_002355295.1_3
17,NZ_CP023465.1,ncRNA,164658,164831,+,"SRR3990749_sra_calls.gff,SRR3990734_sra_calls....","128,129,130,131,132",1.000000,True,5,2,GCA_002355295.1_18
26,NZ_CP023465.1,ncRNA,249532,249705,-,"SRR3990742_sra_calls.gff,SRR3990749_sra_calls....","170,171,172,173,174,175",1.000000,True,6,2,GCA_002355295.1_27
37,NZ_CP023465.1,ncRNA,328878,329088,+,"SRR3990749_sra_calls.gff,SRR3990750_sra_calls....","245,246,247,248,249,250,251,252,253",0.709524,True,9,2,GCA_002355295.1_38
38,NZ_CP023465.1,ncRNA,329138,329442,+,"SRR3990749_sra_calls.gff,SRR3990742_sra_calls....","254,255,256,257,258",0.361842,True,5,2,GCA_002355295.1_39
...,...,...,...,...,...,...,...,...,...,...,...,...
536,NZ_CP023465.1,ncRNA,5655550,5655801,+,"SRR3990734_sra_calls.gff,SRR3990735_sra_calls....","3086,3087,3088,3089,3090,3091",1.000000,True,6,6,GCA_002355295.1_537
537,NZ_CP023465.1,ncRNA,5655862,5656131,+,"SRR3990735_sra_calls.gff,SRR3990733_sra_calls....","3092,3093,3094,3095,3096,3097,3098,3099,3100,3...",1.000000,True,12,2,GCA_002355295.1_538
550,NZ_CP023465.1,ncRNA,5807459,5807607,-,"SRR3990750_sra_calls.gff,SRR3990734_sra_calls....","3178,3179,3180,3181,3182,3183,3184,3185,3186",1.000000,True,9,2,GCA_002355295.1_551
567,NZ_CP023465.1,ncRNA,6074106,6074294,-,"SRR3990749_sra_calls.gff,SRR3990734_sra_calls.gff","3283,3284",1.000000,True,2,2,GCA_002355295.1_568


In [32]:
start = df.loc[:,'start']
end = df.loc[:,'end']

print(start)
print(end)

2        14319
17      164658
26      249532
37      328878
38      329138
        ...   
536    5655550
537    5655862
550    5807459
567    6074106
572    6169153
Name: start, Length: 80, dtype: int64
2        14514
17      164831
26      249705
37      329088
38      329442
        ...   
536    5655801
537    5656131
550    5807607
567    6074294
572    6169291
Name: end, Length: 80, dtype: int64


In [1]:
import sys
from Bio import SeqIO
import pandas as pd

In [8]:
def sequence_cleaner(fasta_file, min_length=50, por_n=100, max_length=500):
    # Create our hash table to add the sequences
    sequences = {}

    # Using the Biopython fasta parse we can read our fasta input
    for seq_record in SeqIO.parse(fasta_file, "fasta"):
        # Take the current sequence
        sequence = str(seq_record.seq).upper()
        # Check if the current sequence is according to the user parameters
        if (
            len(sequence) >= min_length
            and (float(sequence.count("N")) / float(len(sequence))) * 100 <= por_n
            and len(sequence) <= max_length
        ):
            # If the sequence passed in the test "is it clean?" and it isn't in the
            # hash table, the sequence and its id are going to be in the hash
            if sequence not in sequences:
                sequences[sequence] = seq_record.id
            # If it is already in the hash table, we're just gonna concatenate the ID
            # of the current sequence to another one that is already in the hash table
            else:
                sequences[sequence] += "_" + seq_record.id

    # Write the clean sequences

    # Create a file in the same directory where you ran this script
    with open("clear_" + fasta_file, "w+") as output_file:
        # Just read the hash table and write on the file as a fasta format
        for sequence in sequences:
            output_file.write(">" + sequences[sequence] + "\n" + sequence + "\n")

    print("CLEAN!!!\nPlease check clear_" + fasta_file)



In [9]:
fasta_file="/Users/thomasnicholson/phd/RNASeq/srna_seqs/version_1/positive_control.fna"
min_length=50
por_n=100
max_length=500

In [10]:
outFile = open("/Users/thomasnicholson/phd/RNASeq/srna_seqs/version_1/positive_control_filtered.fna", "a")
for seq_record in SeqIO.parse(fasta_file, "fasta"):
    iter += 1
    sequence = str(seq_record.seq).upper()
    if (
            len(sequence) >= min_length
            and (float(sequence.count("N")) / float(len(sequence))) * 100 <= por_n
            and len(sequence) <= max_length
        ):
        seq_name = seq_record.id
        sequence = seq_record.seq
        outFile.write(f">{seq_name}\n{sequence}\n")

TypeError: unsupported operand type(s) for +=: 'builtin_function_or_method' and 'int'

In [11]:
alignDat = pd.read_csv("/Users/thomasnicholson/phd/RNASeq/srna_seqs/version_1/predicted/predicted_genomic_sequence_matches.txt", header=None, delim_whitespace=True)

alignDat = alignDat.iloc[:,[1,3]]
alignDat.columns = ["details", "query_id"]
alignDat[["target_contig", "coord"]] = alignDat.details.str.split("/", expand = True)
alignDat[["target_start", "target_end"]] = alignDat.coord.str.split("-", expand = True)
alignDat = alignDat[["query_id", "target_contig", "target_start", "target_end"]]
alignDat["target_start"] = alignDat["target_start"].astype(str).astype(int)
alignDat['target_start'].dtypes
alignDat["target_end"] = alignDat["target_end"].astype(str).astype(int)
alignDat['target_end'].dtypes
alignDat = alignDat.sort_values(by=['target_start'])
display(alignDat)

NameError: name 'pd' is not defined

In [28]:
target_contigs = alignDat.target_contig.unique()
display(target_contigs)

array(['NC_016824.1', 'NC_004578.1', 'NZ_CP014273.1', 'NC_004741.1',
       'NC_009800.1', 'NC_009801.1', 'NC_012967.1', 'NZ_CP025268.1',
       'NC_016822.1', 'NC_022912.1', 'NZ_CP014272.1', 'NZ_CP022097.1',
       'NZ_CP025515.1', 'NZ_CP016634.1', 'NC_017720.1', 'NC_016823.1',
       'NC_008463.1', 'NC_002516.2', 'NZ_CP018026.1', 'NC_019393.1',
       'NC_016834.1', 'NC_015554.1', 'NC_009786.1', 'NZ_CP003425.1',
       'NC_009791.1', 'NZ_CP025517.1', 'NZ_CP015641.1', 'NC_002947.4',
       'NC_009788.1', 'NZ_CP003424.1', 'NZ_LT855377.1', 'NC_003277.2',
       'NC_009790.1', 'NC_017718.1', 'NC_016833.1', 'NZ_CP025516.1',
       'NC_009787.1', 'NZ_CP015527.1', 'NZ_CP011661.1', 'NZ_LT969519.1',
       'NZ_CP011658.1', 'NC_022899.1', 'NZ_LT905143.1', 'NZ_CP011662.1',
       'NC_021659.1', 'NC_017541.1', 'NZ_LT594096.1', 'NZ_CP019088.1',
       'NZ_CP011961.1', 'NC_006834.1', 'NC_009084.1', 'NZ_CP020358.1',
       'NC_017719.1', 'NC_009083.1', 'NC_017540.1', 'NC_018220.1',
       'NZ_CP016

In [29]:
query_ids = alignDat.query_id.unique()
display(query_ids)

array(['GCA_000283715.1_608', 'GCA_000281215.1_63', 'GCA_002208745.1_77',
       ..., 'GCA_002072655.1_311', 'GCA_002072655.1_312',
       'GCA_002072655.1_314'], dtype=object)

In [56]:
contig = target_contigs[1]
print(contig)
subsetDat = alignDat.loc[alignDat['target_contig'] == contig]
display(subsetDat)

NC_004578.1


,query_id,target_contig,target_start,target_end
97980,GCA_000281215.1_63,NC_004578.1,1,169
143061,GCA_002208745.1_77,NC_004578.1,1,196
143077,GCA_002208745.1_78,NC_004578.1,17,192
8731,GCA_000007565.2_1,NC_004578.1,18,195
94392,GCA_000220485.1_557,NC_004578.1,23240,23737
...,...,...,...,...
236913,GCA_900243355.1_877,NC_004578.1,6393548,6393697
236959,GCA_900243355.1_878,NC_004578.1,6394355,6394504
236971,GCA_900243355.1_879,NC_004578.1,6394567,6394704
237017,GCA_900243355.1_880,NC_004578.1,6395014,6395104


In [51]:
subsetDat['target_start'].dtypes

dtype('int64')

In [52]:
d = {}

3038


In [69]:
def get_overlap_vals(subsetDat, overlaps):
    dat_len = len(subsetDat.index)
    print(dat_len)
    overlapping_ids = []
    lengths = []
    start_val = 0
    end_val = 0
    for i in range(0,dat_len):
        query_val = subsetDat.iloc[i]['query_id']    
        new_start_val = min([subsetDat.iloc[i]['target_start'], subsetDat.iloc[i]['target_end']])
        new_end_val = max([subsetDat.iloc[i]['target_start'], subsetDat.iloc[i]['target_end']])
    #     print(query_val)
    #     print(new_start_val)
    #     print(new_end_val)    
        if end_val > new_start_val:
            overlapping_ids.append(query_val)
            len_1 = end_val - start_val
            len_2 = new_end_val - new_start_val
            shortest_seq = min([len_1, len_2])
            overlap_start = max([start_val, new_start_val])
            overlap_end = min([end_val, new_end_val])
            overlap = (overlap_end - overlap_start)/shortest_seq
            overlaps.append(overlap)
    #         print(overlap)
        else:
            end_val = new_end_val
            start_val = new_start_val
            overlapping_ids = [query_val]
    return(overlaps)
    #     print(overlapping_ids)

In [70]:
overlapping_ids = []
lengths = []
start_val = 0
end_val = 0
overlaps = []
for contig in target_contigs:
    subsetDat =  alignDat.loc[alignDat['target_contig'] == contig]
    overlaps = get_overlap_vals(subsetDat, overlaps)

3
3038
406
9197
8264
7594
8571
8542
10381
9922
8575
3050
5544
3254
181
6
2759
2826
2106
2164
22
2296
156
68
1
60
2211
4241
152
4489
181
181
191
103
435
166
40
141
109
47
1
344
5649
5094
5078
313
4
2471
2698
2464
7
6257
8
6
5600
2083
11
177
19290
2
80
1
8
4537
8474
97
173
5774
5768
5728
5758
2872
2969
2758
2363
5822
2660
3051
12
2121
5079
35
1
13
13
7
1672
14


In [6]:
display(overlaps)

NameError: name 'overlaps' is not defined

In [5]:
import seaborn as sns


ModuleNotFoundError: No module named 'seaborn'